!git clone https://github.com/trucomanx/cnn_face_emotion.git
!mkdir -p OUTPUTS/cnn_face_emotion/cross-validation
!wget https://www.dropbox.com/s/23tutsg2m9o78bp/fer2013.zip
!unzip -n fer2013.zip
!rm fer2013.zip

In [1]:
import os, sys, platform

# Variables

In [2]:
## Seed for the random variables
seed_number=0;



## Training hyperparameters
EPOCAS=50;
BATCH_SIZE=16;

## Model of network
#model_type  = 'mobilenet_v3';
model_type = 'efficientnet_b3'
#model_type = 'inception_v3';
#model_type = 'inception_resnet_v2';
#model_type = 'resnet_v2_50';
#model_type = 'custom1'
#model_type = 'custom_inception'
#model_type = 'custom_residual1'
#model_type = 'custom_dense1'

DATASET_NAME = 'mcfer_v1.0';#'fer2013';


dataset_labels_file = 'training_labels.csv';


# Bibliotecas externas

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime

# Biblioteca local

In [4]:
if os.getenv("COLAB_RELEASE_TAG"):
    sys.path.append('cnn_face_emotion/library');
else:
    sys.path.append('library');

# Uso de GPU

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


# Endereços

In [6]:

## Dataset 
if platform.system()=='Linux':
    if os.getenv("COLAB_RELEASE_TAG"):
        dataset_base_dir_raw = './';
    elif 'microsoft-standard' in platform.uname().release:
        dataset_base_dir_raw = '/mnt/c/Dados/Fernando/DATASET';
    else:
        dataset_base_dir_raw = '/media/fernando/B0EA304AEA300EDA/Dados/Fernando/DATASET';
else:
    dataset_base_dir_raw     = 'C:\\Dados\\Fernando\\DATASET';

if DATASET_NAME=='fer2013':
    RELATIVE_DIR_TRAIN=os.path.join(DATASET_NAME,'archive','train');
    RELATIVE_DIR_TEST =os.path.join(DATASET_NAME,'archive','test');
elif DATASET_NAME=='affectnet':
    RELATIVE_DIR_TRAIN=os.path.join('AffectNet-Sample','input','affectnetsample','train_class');
    RELATIVE_DIR_TEST =os.path.join('AffectNet-Sample','input','affectnetsample','val_class');
elif DATASET_NAME=='mcfer_v1.0':
    RELATIVE_DIR_TRAIN=os.path.join(DATASET_NAME,'archive','train');
    RELATIVE_DIR_TEST =os.path.join(DATASET_NAME,'archive','test');
else:
    print('Error in the dataset name:',DATASET_NAME);
    exit();
'''    
if platform.system()=='Linux':
    if os.getenv("COLAB_RELEASE_TAG"):
        dataset_base_dir     = 'fer2013/archive/train';
        dataset_base_test_dir= 'fer2013/archive/test';
    elif 'microsoft-standard' in platform.uname().release:
        dataset_base_dir     = '/mnt/c/Dados/Fernando/DATASET/fer2013/archive/train';
        dataset_base_test_dir= '/mnt/c/Dados/Fernando/DATASET/fer2013/archive/test';
    else:
        dataset_base_dir     = '/mnt/boveda/DATASETs/FACE-EMOTION/fer2013/archive/train';
        dataset_base_test_dir= '/mnt/boveda/DATASETs/FACE-EMOTION/fer2013/archive/test';
else:
    dataset_base_dir     = 'C:\\Dados\\Fernando\\DATASET\\fer2013\\archive\\train';
    dataset_base_test_dir= 'C:\\Dados\\Fernando\\DATASET\\fer2013\\archive\\test';
'''

dataset_base_dir = os.path.join(dataset_base_dir_raw,RELATIVE_DIR_TRAIN);
dataset_base_test_dir = os.path.join(dataset_base_dir_raw,RELATIVE_DIR_TEST);

print('dataset_base_dir:',dataset_base_dir)
print('dataset_base_test_dir:',dataset_base_test_dir)

## Output
if platform.system()=='Linux':
    if os.getenv("COLAB_RELEASE_TAG"):
        output_base_dir_raw = 'OUTPUTS';
    elif 'microsoft-standard' in platform.uname().release:
        output_base_dir_raw = '/mnt/c/Dados/Fernando/OUTPUTS';
    else:
        output_base_dir_raw = '/media/fernando/B0EA304AEA300EDA/Dados/Fernando/OUTPUTS';
else:
    output_base_dir_raw = 'C:\\Dados\\Fernando\\OUTPUTS';

    
RELATIVE_DIR=os.path.join('cnn_face_emotion_'+DATASET_NAME,'training_holdout');
output_base_dir = os.path.join(output_base_dir_raw,RELATIVE_DIR); 

print('output_base_dir:',output_base_dir)

dataset_base_dir: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/DATASET/mcfer_v1.0/archive/train
dataset_base_test_dir: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/DATASET/mcfer_v1.0/archive/test
output_base_dir: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/OUTPUTS/cnn_face_emotion_mcfer_v1.0/training_holdout


# Print vars

In [7]:
print('model_type:',model_type)
print('    epochs:',EPOCAS)
print('batch-size:',BATCH_SIZE)

model_type: efficientnet_b3
    epochs: 50
batch-size: 16


# Set seed of random variables


In [8]:
np.random.seed(seed_number)
tf.keras.utils.set_random_seed(seed_number);

# Loading data of dataset

In [9]:
# Load filenames and labels
train_data = pd.read_csv(os.path.join(dataset_base_dir,dataset_labels_file));
print(train_data)
# Setting labels
Y = train_data[['label']];
L=np.shape(Y)[0];


FileNotFoundError: [Errno 2] No such file or directory: '/media/fernando/B0EA304AEA300EDA/Dados/Fernando/DATASET/mcfer_v1.0/archive/train/training_labels.csv'

# Setting the cross-validation split


In [ ]:
from sklearn.model_selection import train_test_split
    

training_data, validation_data = train_test_split(train_data, test_size=0.2,shuffle=True, stratify=Y)



# Data augmentation configuration

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

idg    = ImageDataGenerator(rescale=1./255,
                            rotation_range = 10,
                            width_shift_range= 0.07,
                            height_shift_range= 0.07,
                            horizontal_flip=True,
                            shear_range=1.25,
                            zoom_range = [0.75, 1.25],
                            validation_split=0.25
                            )

#idg_val= ImageDataGenerator(rescale=1./255 )

idg_test= ImageDataGenerator(rescale=1./255 )


# Creating output directory

In [ ]:
#output_dir = os.path.join(output_base_dir,'holdout');
output_dir = os.path.join(output_base_dir,'holdout_'+model_type);

try: 
    os.makedirs(output_dir) 
except: 
    pass


# Create new model

In [ ]:
import lib_model as mpp

model, target_size = mpp.create_model('',model_type=model_type);
model.summary()

mpp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats.m'));


# Defining directories

In [ ]:

train_data_generator = idg.flow_from_dataframe(training_data, 
                                               directory = dataset_base_dir,
                                               target_size=target_size,
                                               x_col = "filename", 
                                               y_col = "label",
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical",
                                               shuffle = True);

valid_data_generator  = idg.flow_from_dataframe(validation_data, 
                                                    directory = dataset_base_dir,
                                                    target_size=target_size,
                                                    x_col = "filename", 
                                                    y_col = "label",
                                                    batch_size=BATCH_SIZE,
                                                    class_mode="categorical",
                                                    shuffle = True);

test_data_generator  = idg_test.flow_from_directory(dataset_base_test_dir, 
                                                    target_size=target_size,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True
                                                    );

# Train and validation


In [ ]:
import matplotlib.pyplot as plt


#STEPS_BY_EPOCHS=len(train_data_generator);

# COMPILE NEW MODEL
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy','categorical_crossentropy'])

# CREATE CALLBACKS
best_model_file=os.path.join(output_dir,'model.h5');
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=best_model_file, 
                                                save_weights_only=True,
                                                monitor='val_categorical_crossentropy', 
                                                save_best_only=True, 
                                                verbose=1);

log_dir = os.path.join(output_dir,"logs","fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"));
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# There can be other callbacks, but just showing one because it involves the model name
# This saves the best model
# FIT THE MODEL
history = model.fit(train_data_generator,
                    #steps_per_epoch=STEPS_BY_EPOCHS,
                    epochs=EPOCAS,
                    validation_data=valid_data_generator,
                    callbacks=[checkpoint,tensorboard_callback],
                    verbose=1
                   );


mpp.save_model_history(history,os.path.join(output_dir,"historical.csv"));


# Evaluate best model

In [ ]:
# LOAD BEST MODEL to evaluate the performance of the model
model.load_weights(best_model_file);

results = model.evaluate(test_data_generator)
results = dict(zip(model.metrics_names,results))
print(results,"\n\n");

with open(os.path.join(output_dir,"results_testing.txt"), 'w') as f: 
    for key, value in results.items(): 
        f.write('%s=%s;\n' % (key, value));

tf.keras.backend.clear_session()


In [ ]:

tmp_name='modelo_'+model_type+'.h5';

os.rename(best_model_file,os.path.join(output_dir,tmp_name));